In [ ]:
#!pip install -U datasets
!pip install datasets==2.16.0
!pip install huggingface-hub==0.20.0
!apt-get install -y libsox-dev
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
!pip install causal-conv1d==1.4.0 && pip install mamba-ssm==2.2.2

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Fou

In [ ]:
# Keyword‑Spotting with Mamba SSM
# ------------------------------------------------
# 1. Uses Mamba state‑space blocks instead of a CNN.
# 2. Maintains separate front‑ends for train vs. eval (no masks in eval).
# 3. Handles variable‑length sequences with a custom `collate_fn` (pads –80 dB).
# 4. One‑Cycle LR scheduler stepped correctly each batch.
# 5. Saves best + final checkpoints to Google Drive.


from __future__ import annotations
import json, os, random
from pathlib import Path
from typing import Tuple, Dict
import torch, torchaudio
import torch.nn as nn
import os
import math
import torch.nn.functional as F
import torchvision.models as tvm
import torchaudio.transforms as T
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from tqdm.notebook import tqdm
from mamba_ssm import Mamba

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ---------------------------------------------------------------------
# 2. Waveform-level augmentation (shift + noise)
# ---------------------------------------------------------------------
class WaveToSpec:
    def __init__(self,
                 feature_type: str = "mel",
                 sample_rate: int = 16_000,
                 n_fft: int = 2048,
                 hop_length: int = 256,
                 n_mels: int = 128,
                 n_mfcc: int = 40,
                 top_db: int | None = 80,
                 apply_mask: bool = True,
                 freq_mask_param: int = 15,
                 time_mask_param: int = 10):
        self.feature_type = feature_type.lower(); assert self.feature_type in {"mel","mfcc"}
        self.apply_mask = apply_mask and self.feature_type == "mel"

        if self.feature_type == "mel":
            self.spec = T.MelSpectrogram(sample_rate, n_fft, hop_length, n_mels, power=2)
            self.to_db = T.AmplitudeToDB(stype="power", top_db=top_db)
            if self.apply_mask:
                self.freq_mask = T.FrequencyMasking(freq_mask_param)
                self.time_mask = T.TimeMasking(time_mask_param)
        else:
            self.spec = T.MFCC(sample_rate, n_mfcc,
                                melkwargs=dict(n_fft=n_fft, hop_length=hop_length, n_mels=n_mels))
            self.to_db = None
            self.freq_mask = self.time_mask = None

    def __call__(self, wav: torch.Tensor) -> torch.Tensor:
        if wav.dim() == 1:
            wav = wav.unsqueeze(0)
        feats = self.spec(wav)
        if self.to_db is not None:
            feats = self.to_db(feats.clamp(min=1e-10))
        if self.apply_mask:
            # stronger SpecAugment — apply two freq & two time masks
            feats = self.freq_mask(feats); feats = self.time_mask(feats)
            feats = self.freq_mask(feats); feats = self.time_mask(feats)
        return feats

class Augment:
    def __init__(self, stretch: Tuple[float,float]=(1.0,1.0),
                 shift_ms: int = 100,
                 noise: Tuple[float,float]=(0.,0.005),
                 sr: int = 16_000):
        self.stretch = stretch
        self.shift   = int(shift_ms * sr / 1000)
        self.noise   = noise
        self.sr      = sr

    def _shift(self, x: torch.Tensor):
        if self.shift == 0:
            return x
        s = int(torch.randint(-self.shift, self.shift + 1, ()).item())
        if s == 0:
            return x
        return (F.pad(x, (s, 0))[:, :-s] if s > 0 else F.pad(x, (0, -s))[:, -s:])

    def __call__(self, wav: torch.Tensor):
        squeezed = False
        if wav.dim() == 1:
            wav = wav.unsqueeze(0)
            squeezed = True
        if self.stretch != (1.0, 1.0):
            factor = float(torch.empty(()).uniform_(*self.stretch))
            if abs(factor - 1.0) > 1e-3:
                wav, _ = torchaudio.sox_effects.apply_effects_tensor(
                    wav, self.sr, [["tempo", f"{factor}"]]
                )
        wav = self._shift(wav)
        if self.noise[1] > 0:
            sigma = float(torch.empty(()).uniform_(*self.noise))
            if sigma > 0:
                wav = wav + sigma * torch.randn_like(wav)
        return wav.squeeze(0) if squeezed else wav

In [ ]:
# ---------------------------------------------------------------------
# 3. Dataset wrapper with dataset-level normalization
# ---------------------------------------------------------------------
class SpeechCommands(Dataset):
    def __init__(self, hf_split, aug: Augment | None, frontend: WaveToSpec,
                 wav_len: int = 16_000, mean: float = 0.0, std: float = 1.0):
        self.ds, self.aug, self.front = hf_split, aug, frontend
        self.wav_len = wav_len
        self.mean = mean
        self.std = std

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        sample = self.ds[idx]
        wav = torch.from_numpy(sample["audio"]["array"]).float()

        if wav.numel() < self.wav_len:
            wav = F.pad(wav, (0, self.wav_len - wav.numel()))
        else:
            wav = wav[: self.wav_len]

        if self.aug:
            wav = self.aug(wav)
        feats = self.front(wav)           # [C=1, 40, ~55] for MFCC
        feats = (feats - self.mean) / (self.std + 1e-6)  # Normalize with precomputed stats
        feats = feats.squeeze(0).transpose(0, 1)
        return feats, sample["label"]

In [ ]:
# ---------------------------------------------------------------------
# 4. Helper funcs = collate function + data mean/std + LR decay func
# ---------------------------------------------------------------------
from torch.nn.utils.rnn import pad_sequence

# Add padding to spectrograms if needed
def collate_fn(batch):
    feats, lbls = zip(*batch)
    # Also return true lengths for mask-aware pooling
    lens = torch.tensor([f.size(0) for f in feats], dtype=torch.long)
    feats_padded = pad_sequence(feats, batch_first=True, padding_value=0.0)  # [B, T_max, F]
    return feats_padded, torch.tensor(lbls), lens


# Defines your learning rate schedule
def lr_lambda(step):
    ' ' 'Start with a very low learning rate and gradually increase it' ' '
    if step < warmup_steps:
        return float(step) / float(max(1, warmup_steps))
    progress = float(step - warmup_steps) / float(max(1, total_steps - warmup_steps))
    return max(0.005, 0.5 * (1.0 + math.cos(math.pi * progress)))


# Precompute dataset-level mean and std for MFCCs
def compute_dataset_stats(ds, frontend, wav_len=16_000):
    feats_all = []
    for sample in ds:
        wav = torch.from_numpy(sample["audio"]["array"]).float()
        if wav.numel() < wav_len:
            wav = F.pad(wav, (0, wav_len - wav.numel()))
        else:
            wav = wav[: wav_len]
        feats = frontend(wav).squeeze(0).transpose(0, 1)  # [T, 40]
        feats_all.append(feats)
    feats_all = torch.cat(feats_all, dim=0)
    return feats_all.mean().item(), feats_all.std().item()


In [ ]:
# ---------------------------------------------------------------------
# 5. Mamba KWS model with normalization and residuals
# ---------------------------------------------------------------------
class MambaKWS(nn.Module):
    def __init__(self, num_classes: int, d_model=256, d_state=32, expand=2, n_layers=8, in_ch=1, feature_dim=128):
        super().__init__()

        # Convolutional embedding layer for feature extraction
        self.conv_embed = nn.Sequential(
            nn.Conv2d(in_ch, 32, 3, padding=1),
            nn.BatchNorm2d(32), nn.SiLU(),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32), nn.SiLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64), nn.SiLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64), nn.SiLU(),
            nn.MaxPool2d((2, 1)),
        )

        # Calculate the flattened dimension after convolutions to project to d_model
        freq_dim_after_conv = feature_dim // 4
        flattened_dim = 64 * freq_dim_after_conv

        # Projection layer to map flattened conv features to Mamba's dimension
        self.proj = nn.Sequential(
            nn.Linear(flattened_dim, d_model),
            nn.LayerNorm(d_model),
            nn.SiLU(),
            nn.Dropout(0.1)
        )

        # Add Mamba blocks with layer norm and residuals
        self.blocks = nn.ModuleList([
            nn.ModuleDict({
                "norm": nn.LayerNorm(d_model),
                "mamba": Mamba(d_model=d_model, d_state=d_state, expand=expand),
                "dropout": nn.Dropout(max(0.02, 0.05 - (i * 0.005)))
            }) for i in range(n_layers)
        ])
        self.pre_classifier_norm = nn.LayerNorm(d_model)

        # Classifier head
        self.classifier_dropout = nn.Dropout(0.1)
        self.classifier = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.SiLU(),
            nn.Dropout(0.05),
            nn.Linear(d_model // 2, num_classes)
        )

    # accept lengths for mask-aware pooling
    def forward(self, x, lengths: torch.Tensor | None = None):  # x: [B, T, F]
        # reshape for Conv2d: [B, T, F] -> [B, 1, F, T]
        x = x.permute(0, 2, 1).unsqueeze(1)

        # conv front-end
        x = self.conv_embed(x)                  # [B, 64, F', T']

        # flatten per time-step and project
        x = x.permute(0, 3, 1, 2).contiguous().flatten(2)  # [B, T', 64*F']
        x = self.proj(x)                                   # [B, T', d_model]

        for i, blk in enumerate(self.blocks):
            residual = x
            x = blk["norm"](x)
            x = blk["mamba"](x)
            x = blk["dropout"](x)
            x = residual + x

        x = self.pre_classifier_norm(x)

        # mask-aware mean pooling over time
        if lengths is not None:
            t_lens = torch.div(lengths, 2, rounding_mode='floor').clamp(min=1).to(x.device)  # first pool halves time
            Tprime = x.size(1)
            mask = (torch.arange(Tprime, device=x.device)[None, :] < t_lens[:, None]).float()  # [B, T']
            mask = mask.unsqueeze(-1)  # [B, T', 1]
            x_sum = (x * mask).sum(dim=1)                            # [B, d_model]
            denom = mask.sum(dim=1).clamp(min=1.0)                   # [B, 1]
            pooled = x_sum / denom
        else:
            pooled = x.mean(dim=1)

        # single-head output (no aux)
        main_output = self.classifier(self.classifier_dropout(pooled))
        return main_output

# ---------------------------------------------------------------------
@torch.no_grad()
def evaluate(model, loader, device, criterion):
    model.eval()
    tot = correct = loss_sum = 0
    for batch in loader:
        xb, yb, lb = batch
        xb, yb, lb = xb.to(device), yb.to(device), lb.to(device)
        logits = model(xb, lengths=lb)
        loss = criterion(logits, yb)
        loss_sum += loss.item() * xb.size(0)
        correct += (logits.argmax(1) == yb).sum().item()
        tot += xb.size(0)
    return loss_sum / tot, 100 * correct / tot

In [ ]:
# ---------------------------------------------------------------------
# 6. Main script
# ---------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    use_amp = (device.type == "cuda")

    # ---- dataset
    ds = load_dataset("google/speech_commands", "v0.02")
    n_classes = len(ds["train"].features["label"].names)

    # log-mel + SpecAugment (train only)
    feature_type = "mel"        # "mel"/"mfcc"
    Epochs = 100
    base_lr = 5e-4
    warmup_frac = 0.12          # % of epochs warmup

    frontend_train = WaveToSpec(
        feature_type=feature_type,
        n_mfcc=40, n_mels=128,
        apply_mask=True,         # SpecAugment on train
        freq_mask_param=12,
        time_mask_param=20
    )
    frontend_eval = WaveToSpec(
        feature_type=feature_type,
        n_mfcc=40, n_mels=128,
        apply_mask=False
    )

    frontend_stats = WaveToSpec(feature_type=feature_type, n_mfcc=40, n_mels=128, apply_mask=False)

    # Waveform augs shift + a bit of noise
    aug = Augment(shift_ms=120, noise=(0., 0.005))

    # Normalization stats
    train_mean, train_std = compute_dataset_stats(ds["train"], frontend_stats)

    # Datasets
    train_ds = SpeechCommands(ds["train"], aug, frontend_train, mean=train_mean, std=train_std)
    val_ds   = SpeechCommands(ds["validation"], None, frontend_eval, mean=train_mean, std=train_std)
    test_ds  = SpeechCommands(ds["test"], None, frontend_eval, mean=train_mean, std=train_std)

    # Loaders
    dl_kwargs = dict(
        batch_size=64,
        num_workers=2,
        pin_memory=True,
        persistent_workers=True,
        collate_fn=collate_fn
    )
    train_dl = DataLoader(train_ds, shuffle=True, **dl_kwargs)
    val_dl   = DataLoader(val_ds, shuffle=False, **dl_kwargs)

    # Model
    model = MambaKWS(n_classes, d_model=64, d_state=16, n_layers=8).to(device)

    # Loss/opt/sched (per-batch schedule; short warmup; cosine with floor)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.07)
    opt = torch.optim.AdamW(model.parameters(), lr=base_lr, weight_decay=1.8e-4, betas=(0.9, 0.999))

    steps_per_epoch = len(train_dl)
    total_steps     = steps_per_epoch * Epochs
    warmup_steps    = int(total_steps * warmup_frac)
    sched = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda)


    # Training loop with AMP, clipping, collapse guard, and checkpoints
    best_val_acc = 0.0
    prev_val_acc = 0.0
    BEST_PATH = Path("/content/best_kws.pt")
    BEST_PATH.parent.mkdir(parents=True, exist_ok=True)

    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
    global_step = 0  # track steps to know when warmup ends

    for epoch in range(1, Epochs + 1):
        model.train()
        running_loss = correct = total = 0.0

        pbar = tqdm(train_dl, desc=f"Epoch {epoch:02d}")
        for batch in pbar:
            xb, yb, lb = batch
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            lb = lb.to(device, non_blocking=True)

            with torch.amp.autocast(device_type=device.type, dtype=torch.bfloat16,
                                    enabled=use_amp and torch.cuda.is_bf16_supported()):
                if torch.isnan(xb).any():
                    xb = torch.nan_to_num(xb, nan=0.0)

                logits = model(xb, lengths=lb)
                loss = criterion(logits, yb)

                if not torch.isfinite(loss):
                    continue

            opt.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.3)
            scaler.step(opt)
            scaler.update()
            sched.step()  # per-batch warmup + cosine
            global_step += 1

            pred = logits.argmax(1)
            correct += (pred == yb).sum().item()
            total += yb.size(0)
            running_loss += loss.item() * yb.size(0)

            pbar.set_postfix(
                train_loss=f"{running_loss / max(1,total):.3f}",
                train_acc=f"{100 * correct / max(1,total):.2f}%",
                lr=f"{opt.param_groups[0]['lr']:.2e}"
            )

        tr_acc = 100.0 * correct / max(1, total)
        val_loss, val_acc = evaluate(model, val_dl, device, criterion)
        print(f"Epoch {epoch:02d} ➜ train {tr_acc:.2f}% | val {val_acc:.2f}% (loss {val_loss:.3f}) | lr {opt.param_groups[0]['lr']:.2e}")

        # Collapse guard: big sudden drop -> reload best + shrink LR
        if epoch > 1 and prev_val_acc > 50.0 and val_acc < 0.5 * prev_val_acc:
            print(f"WARNING: accuracy collapse ({prev_val_acc:.2f}% → {val_acc:.2f}%). Restoring best and reducing LR ×5.")
            if BEST_PATH.exists():
                model.load_state_dict(torch.load(BEST_PATH, map_location=device))
            for g in opt.param_groups:
                g['lr'] = max(g['lr'] / 5.0, 1e-6)
            print(f"New LR: {opt.param_groups[0]['lr']:.2e}")

        # Best-by-accuracy checkpoint
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), BEST_PATH)
            print(f"** Saved new best model params ** @ {best_val_acc:.2f}%")

        prev_val_acc = val_acc



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/datasets/load.py:1429: FutureWarning: The repository for google/speech_commands contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/speech_commands
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets

Generating train split:   0%|          | 0/84848 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9982 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4890 [00:00<?, ? examples/s]

/tmp/ipython-input-2921285276.py:75: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)


...

Epoch 98 ➜ train 97.88% | val 97.34% (loss 0.590) | lr 2.50e-06


Epoch 99:   0%|          | 0/1326 [00:00<?, ?it/s]

Epoch 99 ➜ train 97.93% | val 97.37% (loss 0.590) | lr 2.50e-06


Epoch 100:   0%|          | 0/1326 [00:00<?, ?it/s]

Epoch 100 ➜ train 97.92% | val 97.33% (loss 0.590) | lr 2.50e-06


In [ ]:
# --- Save LAST params locally + Drive ---
torch.save(model.state_dict(), "/content/last_kws_mamba_noAux.pt")

CKPT_DIR = "/content/drive/MyDrive/kws_models_noAux_64_8"
os.makedirs(CKPT_DIR, exist_ok=True)
torch.save(model.state_dict(), f"{CKPT_DIR}/last_kws_mamba_noAux.pt")
print("Saved LAST model to Drive")

# --- Copy BEST (by val_acc) to Drive if it exists ---
best_local = "/content/best_kws.pt"
if os.path.exists(best_local):
    import shutil
    dst = f"{CKPT_DIR}/best_kws.pt"
    shutil.copy2(best_local, dst)          # copies the already-saved BEST
    print(f"Copied BEST model (val_acc={best_val_acc:.2f}%) to Drive: {dst}")
else:
    print("WARNING: no best checkpoint was found to copy.")

Saved LAST model to Drive
Copied BEST model (val_acc=97.42%) to Drive: /content/drive/MyDrive/kws_models_noAux_64_8/best_kws.pt
